🛡️ Project Summary: Fraud Model Diagnostics
I've successfully monitored Poundbank's fraud detection model using the nannyml library to diagnose the recent drop in accuracy. My analysis focused on tracking performance alerts and identifying the source of data drift.

🔍 What I Found (The Results)
My findings confirm that both model performance has degraded and the underlying data patterns have fundamentally changed.

1. Model Performance Alerts (Task 1) 📉

I used Confidence-Based Performance Estimation (CBPE) to track the model's accuracy monthly. I found that both the estimated (expected) and realized (actual) accuracy dropped below the acceptable confidence band during these periods:

Months with Performance Alerts (months_with_performance_alerts):

Result: A list of months in the format ["month_year"].

2. Highest Data Drift Feature (Task 2) 📈

I calculated the univariate drift for all features using the recommended Kolmogorov-Smirnov and Chi-square/G-statistic tests. The feature showing the most extreme change is the one most likely responsible for destabilizing the model:

Highest Correlation Feature (highest_correlation_feature):

Result: The name of the single feature that exhibited the highest drift score.

3. Data Quality Alert: Transaction Amount (Task 3) 💰

I monitored the average value of transactions to detect unusual financial activity, confirming that a key transactional metric deviated significantly from historical norms:

Alert Average Transaction Amount (alert_avg_transaction_amount):

Result: A single floating-point number representing the high average amount that triggered the alert, rounded to at least one decimal place.

💡 Analysis: Why the Model Broke (The Extra Task)
My drift analysis points to a significant behavioral change by fraudsters, which is the root cause of the accuracy drop.

Observation: I saw two major shifts starting in Spring: the vanishing of transactions made within one minute of login, and a simultaneous increase in the average transaction amount.

Conclusion (Hypothesis of Adversarial Drift): My conclusion is that fraudsters have adapted their strategy to evade detection. They realized that rapid transactions were likely being flagged by the old model. They are now:

Waiting longer after logging in to execute transactions, thereby bypassing the model's original time-based vulnerability.

Consolidating their theft into fewer, larger transfers to maximize efficiency and minimize exposure to detection.

This adaptation created new data patterns (drift) that the old model was never trained to handle, leading directly to the performance collapse I observed.

In [2]:
# 1. Install nannyml
!pip install --upgrade nannyml

# 2. Downgrade NumPy to a widely compatible version (NumPy 1.26.x)
# This prevents the "np.NaN was removed" error.
!pip install numpy==1.26.4

# 3. Import and disable logging
import pandas as pd
import nannyml as nml
import numpy as np
nml.disable_usage_logging()

In [3]:
# Fetching the files from gdrive
from google.colab import drive

# Mount Google Drive
print("Mounting Google Drive...")
drive.mount('/content/drive')

file_path1 = '/content/drive/MyDrive/Datacamp_monitor fraud detection model files/reference.csv'
reference = pd.read_csv(file_path1)

file_path2 = '/content/drive/MyDrive/Datacamp_monitor fraud detection model files/analysis.csv'
analysis = pd.read_csv(file_path2)

# Show first 5 rows of reference dataset
print("Reference dataset head:")
print(reference.head())

# Show info summary of reference dataset
print("\nReference dataset info:")
print(reference.info())

# Show first 5 rows of analysis dataset
print("\nAnalysis dataset head:")
print(analysis.head())

# Show info summary of analysis dataset
print("\nAnalysis dataset info:")
print(analysis.info())

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Reference dataset head:
                 timestamp  time_since_login_min  transaction_amount  \
0  2018-01-01 00:00:00.000              1.561750              3981.1   
1  2018-01-01 00:08:43.152              1.658074              1267.9   
2  2018-01-01 00:17:26.304              2.454287              1984.7   
3  2018-01-01 00:26:09.456              2.392085              2265.2   
4  2018-01-01 00:34:52.608              2.189806              2126.8   

  transaction_type  is_first_transaction  user_tenure_months  is_fraud  \
0          PAYMENT                 False            0.318980       1.0   
1          PAYMENT                 False            7.391323       0.0   
2          CASH-IN                 False            0.781225       1.0   
3         CASH-OUT                 False            0.680473       1.0   
4          CASH-IN 

In [4]:
## Identifing the months when both the estimated and realized ROC AUC of the model have alerts. Store the names of these months as lowercase strings in a list named months_with_performance_alerts.

# Get the estimated performance using CBPE algorithm
cbpe = nml.CBPE(
    timestamp_column_name="timestamp",
    y_true="is_fraud",
    y_pred="predicted_fraud",
    y_pred_proba="predicted_fraud_proba",
    problem_type="classification_binary",
    metrics=["accuracy"],
    chunk_period="m"
)

cbpe.fit(reference)
est_results = cbpe.estimate(analysis)

# Calculate the realized performance
calculator = nml.PerformanceCalculator(
    y_true="is_fraud",
    y_pred="predicted_fraud",
    y_pred_proba="predicted_fraud_proba",
    timestamp_column_name="timestamp",
    metrics=["accuracy"],
    chunk_period="m",
    problem_type="classification_binary",
)
calculator = calculator.fit(reference)
calc_results = calculator.calculate(analysis)

# Compare the results and find the months with alerts
est_results.compare(calc_results).plot().show()
months_with_performance_alerts = ["april_2019", "may_2019", "june_2019"]
print(months_with_performance_alerts)

## Determining which alerting feature has the strongest correlation with the model’s realized performance. Store the name of this feature in a variable named highest_correlation_feature.

features = ["time_since_login_min", "transaction_amount",
            "transaction_type", "is_first_transaction",
            "user_tenure_months"]

# Calculate the univariate drift results
udc = nml.UnivariateDriftCalculator(
    timestamp_column_name="timestamp",
    column_names=features,
    chunk_period="m",
    continuous_methods=["kolmogorov_smirnov"],
    categorical_methods=["chi2"]
)

udc.fit(reference)
udc_results = udc.calculate(analysis)

# Use the correlation ranker
ranker = nml.CorrelationRanker()
ranker.fit(
    calc_results.filter(period="reference"))

correlation_ranked_features = ranker.rank(udc_results, calc_results)

# Find the highest correlating feature
display(correlation_ranked_features)
highest_correlation_feature = "time_since_login_min"
print(highest_correlation_feature)

## Use the summary average statistics calculator to find out what were the monthly average transactions amounts, and if there's any alert. Record this value in a variable called alert_avg_transaction_amount.

# Calculate average monthly transactions
calc = nml.SummaryStatsAvgCalculator(
    column_names=["transaction_amount"],
    chunk_period="m",
    timestamp_column_name="timestamp",
)

calc.fit(reference)
stats_avg_results = calc.calculate(analysis)

# Find the month
stats_avg_results.plot().show()
alert_avg_transaction_amount = 3069.8184
print(alert_avg_transaction_amount)

## Answer to the bonus question
"""
First, I recommend looking at the distribution plots for all features and analyzing them using this command:
- `univariate_data_drift.filter(column_names=features).plot(kind="distribution")`

Observations:

- time_since_log_min - From April to June, the transactions made within one minute after logging in completely vanished.
- transaction_amount - In May and June, a larger number of transactions appeared. Additionally, as you discovered in the third question, the average transaction value has increased and raised an alert.

Possible explanation:

Fraudsters may have noticed that early card transactions, when done right after logging in, often led to account blocking. As a result, they began waiting a bit longer before transferring money to their account to avoid detection. Furthermore, they tend to make a single larger transfer instead of many smaller ones, leading to an increase in the average transaction value.
"""

/usr/local/lib/python3.12/dist-packages/nannyml/chunk.py:294: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'M' instead.
  grouped_data = data.groupby(pd.to_datetime(data[self.timestamp_column_name]).dt.to_period(self.offset))
/usr/local/lib/python3.12/dist-packages/nannyml/chunk.py:294: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'M' instead.
  grouped_data = data.groupby(pd.to_datetime(data[self.timestamp_column_name]).dt.to_period(self.offset))
/usr/local/lib/python3.12/dist-packages/nannyml/chunk.py:294: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'M' instead.
  grouped_data = data.groupby(pd.to_datetime(data[self.timestamp_column_name]).dt.to_period(self.offset))
/usr/local/lib/python3.12/dist-packages/nannyml/chunk.py:294: FutureWarning: 'm' is deprecated and will be removed in a future version, please use 'M' instead.
  grouped_data = data.groupby(pd.to_dateti

/usr/local/lib/python3.12/dist-packages/nannyml/chunk.py:294: FutureWarning:

'm' is deprecated and will be removed in a future version, please use 'M' instead.

/usr/local/lib/python3.12/dist-packages/nannyml/chunk.py:294: FutureWarning:

'm' is deprecated and will be removed in a future version, please use 'M' instead.



['april_2019', 'may_2019', 'june_2019']


/usr/local/lib/python3.12/dist-packages/nannyml/chunk.py:294: FutureWarning:

'm' is deprecated and will be removed in a future version, please use 'M' instead.

/usr/local/lib/python3.12/dist-packages/nannyml/chunk.py:294: FutureWarning:

'm' is deprecated and will be removed in a future version, please use 'M' instead.

/usr/local/lib/python3.12/dist-packages/nannyml/chunk.py:294: FutureWarning:

'm' is deprecated and will be removed in a future version, please use 'M' instead.

/usr/local/lib/python3.12/dist-packages/nannyml/chunk.py:294: FutureWarning:

'm' is deprecated and will be removed in a future version, please use 'M' instead.

/usr/local/lib/python3.12/dist-packages/nannyml/chunk.py:294: FutureWarning:

'm' is deprecated and will be removed in a future version, please use 'M' instead.

/usr/local/lib/python3.12/dist-packages/nannyml/drift/ranker.py:376: FutureWarning:

Series.ravel is deprecated. The underlying array is already 1D, so ravel is not necessary.  Use `to_numpy

,column_name,pearsonr_correlation,pearsonr_pvalue,has_drifted,rank
0,time_since_login_min,0.952925,1.045775e-09,True,1
1,transaction_amount,0.626235,5.427712e-03,True,2
2,is_first_transaction,0.054255,8.306916e-01,True,3
3,user_tenure_months,-0.100547,6.913911e-01,True,4
4,transaction_type,-0.186569,4.585328e-01,True,5


time_since_login_min


/usr/local/lib/python3.12/dist-packages/nannyml/chunk.py:294: FutureWarning:

'm' is deprecated and will be removed in a future version, please use 'M' instead.

/usr/local/lib/python3.12/dist-packages/nannyml/chunk.py:294: FutureWarning:

'm' is deprecated and will be removed in a future version, please use 'M' instead.

/usr/local/lib/python3.12/dist-packages/plotly/io/_json.py:560: UserWarning:

Discarding nonzero nanoseconds in conversion.



3069.8184


'\nFirst, I recommend looking at the distribution plots for all features and analyzing them using this command: \n- `univariate_data_drift.filter(column_names=features).plot(kind="distribution")`\n\nObservations:\n\n- time_since_log_min - From April to June, the transactions made within one minute after logging in completely vanished.\n- transaction_amount - In May and June, a larger number of transactions appeared. Additionally, as you discovered in the third question, the average transaction value has increased and raised an alert.\n\nPossible explanation: \n\nFraudsters may have noticed that early card transactions, when done right after logging in, often led to account blocking. As a result, they began waiting a bit longer before transferring money to their account to avoid detection. Furthermore, they tend to make a single larger transfer instead of many smaller ones, leading to an increase in the average transaction value.\n'